In [1]:
import pandas as pd

# Specify the file path
file_path = "../data/Steel_industry.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

C:\Users\Lucas\AppData\Local\Temp\ipykernel_9076\508217193.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Date_Time,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_Of_Week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [2]:
df.drop(["Date_Time", "WeekStatus", "Day_Of_Week"], axis=1, inplace=True)
class_mapping = {'Light_Load': 1, 'Medium_Load': 2, 'Maximum_Load': 3}
df['Load_Type'] = df['Load_Type'].map(class_mapping)

In [3]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical columns
numerical_columns = ['Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor',
                     'NSM']

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Scale the numerical columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [7]:
# Set the desired number of data points per class
desired_num_data = 50

# Get the counts of each class
class_counts = df['Load_Type'].value_counts()

# Get the names of the 3 most present classes
top_classes = class_counts.nlargest(3).index.tolist()

# Initialize an empty DataFrame to store the sampled data
sampled_df = pd.DataFrame()

# Iterate over the top classes
for class_label in top_classes:
    # Get the data points for the current class
    class_data = df[df['Load_Type'] == class_label]

    # Sample the desired number of data points for the current class
    sampled_data = class_data.sample(n=min(desired_num_data, len(class_data)), random_state=42)

    # Append the sampled data to the sampled_df DataFrame
    sampled_df = pd.concat([sampled_df, sampled_data])

# Update the df DataFrame with the sampled data
df = sampled_df

In [9]:
# Define the file path for the .txt file
txt_file_path = "../data/steel_industry.txt"

# Open the .txt file in write mode
with open(txt_file_path, 'w') as file:
    # Write the X line
    x_line = "X=["
    x_line += "; ".join([" ".join(map(str, row)) for row in df[numerical_columns].values])
    x_line += "]\n"
    file.write(x_line)

    # Write the Y line
    y_line = "Y = Vector{Any}([" + ", ".join(map(str, df['Load_Type'].values)) + "])"
    file.write(y_line)